In [1]:
import getpass
import os

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = getpass.getpass()

KeyboardInterrupt: Interrupted by user

In [1]:
from langchain_core.documents import Document
documents = [
    Document(
        page_content = "Dogs are great companions, known for their loyalty and friendliness.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Cats are independent pets that often enjoy their own space.",
        metadata={"source": "mammal-pets-doc"},
    ),
]

In [2]:
from langchain_community.document_loaders import PyPDFLoader

file_path = "nke-10k-2023.pdf"
loader = PyPDFLoader(file_path)
docs = loader.load()
print(len(docs))

107


In [3]:
print(f"{docs[0].page_content[:200]}\n")
print(docs[0].metadata)

Table of Contents
UNITED STATES
SECURITIES AND EXCHANGE COMMISSION
Washington, D.C. 20549
FORM 10-K
(Mark One)
☑  ANNUAL REPORT PURSUANT TO SECTION 13 OR 15(D) OF THE SECURITIES EXCHANGE ACT OF 1934
F

{'producer': 'EDGRpdf Service w/ EO.Pdf 22.0.40.0', 'creator': 'EDGAR Filing HTML Converter', 'creationdate': '2023-07-20T16:22:00-04:00', 'title': '0000320187-23-000039', 'author': 'EDGAR Online, a division of Donnelley Financial Solutions', 'subject': 'Form 10-K filed on 2023-07-20 for the period ending 2023-05-31', 'keywords': '0000320187-23-000039; ; 10-K', 'moddate': '2023-07-20T16:22:08-04:00', 'source': 'nke-10k-2023.pdf', 'total_pages': 107, 'page': 0, 'page_label': '1'}


In [4]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)
all_splits = text_splitter.split_documents(docs)
len(all_splits)

516

In [6]:
pip install -qU langchain-ollama

Note: you may need to restart the kernel to use updated packages.


In [5]:
from langchain_ollama import OllamaEmbeddings
embeddings = OllamaEmbeddings(model="llama3")

In [6]:
vector_1 = embeddings.embed_query(all_splits[0].page_content)
vector_2 = embeddings.embed_query(all_splits[1].page_content)

assert len(vector_1) == len(vector_2)
print(f"Generated vectors of length {len(vector_1)}\n")
print(vector_1[:10])

Generated vectors of length 4096

[-0.0043087127, -0.029127879, -0.01385499, 0.0040667765, -0.005861809, -0.02568284, -0.0069189444, -0.006874306, -0.034442145, -0.00085926877]


In [9]:
pip install -qU langchain-core

Note: you may need to restart the kernel to use updated packages.


In [8]:
from langchain_core.vectorstores import InMemoryVectorStore
vector_store = InMemoryVectorStore(embeddings)

In [ ]:
ids = vector_store.add_documents(documents=all_splits)

In [ ]:
results = vector_store.similarity_search(
    "How many distribution centers does Nike have in the US?"
)
print(results[0])

In [ ]:
results = await vector_store.asimilarity_search("When was Nike incorporated?")
print(results[0])

In [ ]:
results = vector_store.similarity_search_with_score("What was Nike's revenue in 2023?")
doc,score = results[0]
print(f"Score: {score}\n")
print(doc)

In [ ]:
embedding = embeddings.embed_query("How were Nike's margins impacted in 2023?")
results = vector_store.similarity_search_by_vector(embedding)
print(results[0])

In [ ]:
from typing import List
from langchain_core.documents import Document
from langchain_core.runnables import chain

@chain
def retriever(query:str) -> List[Document]:
    return vector_store.similarity_search(query,k=1)

retriever.batch(
    [
        "How many distribution centers does Nike have in the US?",
        "When was Nike incorporated?",
    ],
)

In [ ]:
retriever = vector_store.as_retriever(
    search_type="similarity",
    search_kwargs={"k":1},
)
retriever.batch(
    [
        "How many distribution centers does Nike have in the US?",
        "When was Nike incorporated?",
    ],
)